# Libraries import

### Python libraries

In [1]:
import io
import os
import json
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime

from scipy.interpolate import interp1d

from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras.metrics import MeanAbsolutePercentageError

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Conv1D, Lambda
from tensorflow.keras.optimizers import Adam


### R libraries

In [2]:
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.rinterface


rpy2.robjects.numpy2ri.activate()

base = importr('base')
Factoextra = importr("factoextra")


# Time series class

A class representing the time series and every information needed to work with.
Contains functions to discretize it, to find the optimal number of clusters, the optimal time step and the models window size.
Contains the models used for the hybrid prediction, functions to calculate and adjust the weights that correspond to the importance of the models for the prediction of the final values.

In [3]:

class TimeSeries():
    
    """
        A class representing the time series and every information needed to work with it
        Contains functions to discretize it, finds the optimal number of clusters, the optimal 
        time step and the models window size
        Contains the models used for the hybrid prediction, functions to calculate and adjust
        the weights that correspond to the importance of the models for the prediction of the final values
        
        @Attributes:
        conf: dict
            The configuration of the time series where every useful information is stored 
            like the optimal number of cluster, the time step, the device code, 
            the optimal window size for each model...
        df: DataFrame
            The DataFrame obtained after reading the data files
        ts: array of float
            The raw time series
        ts_dis: array of float
            The discretized time series
        trainX: array of float
            The training inputs
        trainY: array of float
            The training labels/output
        testX: array of float
            The testing inputs
        testY:
            The testing labels/output
        models: dict
            Dictionary containing the models used for predicting the time series, keys are the models names
        predict: DataFrame
            DataFrame that contains every prediction for each model and for each window in the test set
            Contains the final prediction computed with the weights
        error: DataFrame
            DataFrame that contains the MAE computed between the prediction of each model and the original test set
        weights: array of float
            The weights of the models that correspond to their importance for the prediction of the final values 
        last_values: DataFrame
            Last values used by the weights calculation function for the standard deviation 
    """
    
    
    def __init__(self, device_code=None, time_step=None, load_fileconf=None, load_dictconf=None, 
                 nbr_of_files=1000, threshold=15):
        
        """
            @Params:
            device_code: str
                The device code for the appliance to study, 
                useless if arguments load_fileconf or load_dictconf aren't None
            time_step: int
                The time step to use when reading the files, represents a time step in minutes, 
                if left None, the time step will be computed automatically
            load_fileconf: str
                The filename to load a configuration from a json stored locally in a text file
            load_dictconf: dict
                Load a configuration from a dictionary
            nbr_of_files: int
                The number of files to read
            threshold: int
                Threshold representing the limit while computing the MAPE 
                between raw time series and scaled time series
        """
        
        if load_fileconf:
            self.load_conf(load_fileconf)
          
        elif load_dictconf:
            self.conf = load_dictconf
            
        else:   
            self.conf = {
                "device": device_code
            }
        
            if time_step:
                self.conf["time_step"] = time_step  
                
            else:
                self.conf["time_step"] = find_time_step(device_code, threshold=threshold)
            
        self.df = create_dataframe(self.conf["device"], self.conf["time_step"], nbr_of_files=nbr_of_files)
        self.df = self.df.dropna()
        self.ts = self.df["active_power"]
        
    
    def set_nb_clusters(self):
        
        """
            Finds the optimal number of clusters needed to discretize the time series
            Uses the fviz_nbclust() from Factoextra in an embedded R code to compute it
        """
        
        print("Finding the optimal number of clusters...")
 
        sample = ro.r.matrix(self.df[self.df["filename"].between(1, 4)]["active_power"].to_numpy())
                
        r=ro.r("""
            check = function(matrix) {
            n_clust = fviz_nbclust(matrix, kmeans, k.max = 15)

            n_clust = n_clust$data

            max_cluster = as.numeric(n_clust$clusters[which.max(n_clust$y)])
            return(max_cluster)
            }
        """)

        result = r(sample)
        self.conf["nb_clust"] = int(result[0])
        
        print(f"Optimal number of clusters is {self.conf['nb_clust']}\n")
                    
            
    def discretize(self, threshold=100):
        
        """
            Discretizes the time series using the number of clusters in the configuration dictionary
            Computes the mape between the original time series and the discretized one
            If the mape is too high, the orginal time series will be used instead of the discretized one 
            
            @Params:
            threshold: int
                The maximum threshold not to be exceeded for the discretization

        """
        
        print("Starting discretization...")
        
        if not "nb_clust" in self.conf.keys():
            print("No number of clusters assigned\n")
            self.set_nb_clusters()
        
        self.ts_dis = clustering(self.ts, self.conf["nb_clust"])
        
        error = MAPE(self.ts, self.ts_dis)
        print(error)
        
        if error > threshold:
            self.ts_dis = self.ts
            print("\nError is too high, original time series will be used instead of disctretized one\n")
            
        print("Discretization done\n")
        
         
    def plot_series(self, t1=0, t2=100, t1p=None, t2p=None):
        
        """
            Plots the raw time series and the discretized one
            
            @Params:
            t1: int
                The lower bound index when plotting the time series
            t2: int
                The upper bound index when plotting the time series
            t1p: float between 0 and 1
                The lower bound index in percentage when plotting 
            t2p: float between 0 and 1
                The upper bound of time in percentage when plotting
        """
        
        plot_discretized(self.ts, self.ts_dis, t1=t1, t2=t2, t1p=t1p, t2p=t2p)
        
        
    def find_w_size(self, name, output_width=30, min_w=10, max_w=75, step=5, offset=0, 
                    sum_=False, window=None):
        
        """
            Finds the best input window size for a given model using an interval of values
            Computes the loss over windows size and keeps the size with the minimum error on the validation set
            
            @Params:
            name: str
                The name of the model used for the prediction, allowed names are 'CNN_LSTM', 'CNN', 'LSTM' and 'MLP'
            output_width: int
                The output window size
            min_w: int
                The minimum value of the interval to find the best window size
            max_w: int
                The maximum value of the interval to find the best window size
            step: int
                The step of values inside the interval
            offset: int
                The offset between the last value of the input window and the first value of the output window
            sum_: bool
                If true, the output will be the sum over the next 30 minutes
            window: int
                Optional input window size, if used, the function won't find the best window size and will use 
                this value instead
                
            @Returns:
            window_size: int
                The optimal window size for the given model
        """
        
        loss_arr = []
        val_loss_arr = []
        
        print("Finding best window size..")
        print(f"Model: {name}, output size: {output_width}\n")
        
        if window:
            
            self.create_train_test(name=name, f_size=window, offset=offset, output_width=output_width, sum_=sum_)
            model, loss, val_loss = get_model(name, self.trainX, self.trainY)
        
        else:
            
            for i in range(min_w, max_w, step):
            
                self.create_train_test(name=name, f_size=i, offset=offset, output_width=output_width, sum_=sum_)
                model, loss, val_loss = get_model(name, self.trainX, self.trainY)
                
                print(f"For window of {i} values, MAPE = {loss}")
                loss_arr.append(loss)
                val_loss_arr.append(val_loss)
                
                temp = np.insert(val_loss_arr, 0, val_loss_arr[0])
                temp = np.append(temp, val_loss_arr[-1])
            
                smooth = np.convolve(temp, [1, 2, 1], mode='valid')
     
                if (len(smooth)-np.argmin(smooth)) > 4:
                    break
                
            print("Done")
            
            val_loss_arr = np.insert(val_loss_arr, 0, val_loss_arr[0])
            val_loss_arr = np.append(val_loss_arr, val_loss_arr[-1])
            val_loss_arr_smooth = np.convolve(val_loss_arr, [1, 2, 1], mode='valid')     
            
            idx = np.argmin(val_loss_arr_smooth)
            
            window_size = range(min_w, max_w, step)[idx]
            
            range_ = range(min_w, max_w, step)[:len(loss_arr)]
            plt.plot(range_, loss_arr, label="loss", color="#33638DFF")
            plt.plot(range_, val_loss_arr[1:-1], label="val_loss", color="#3CBB75FF")
            plt.plot(range_, val_loss_arr_smooth/4, 
                     label="smooth_val_loss", color="#d18756")
            
            plt.axvline(x=window_size, linestyle="--", c="black", lw=1)
            plt.legend()
            plt.title(name + " model")
            plt.xlabel("window size")
            plt.ylabel("loss")
            plt.show()
            
            print(f"Best window size for {name} is {window_size}\n")

            return window_size
    
    
    def create_train_test(self, name, f_size=10, offset=0, output_width=1, train_size=0.8, sum_=False):
        
        """
            Creates a train and a test set for the deep learning models
            
            @Params:
            name: str
                The name of the model used for the prediction, allowed names are 'CNN_LSTM', 'CNN', 'LSTM' and 'MLP'
            f_size: int
                The input window size
            offset: int
                The offset between the last value of the input window and the first value of the output window
            output_width:
                The output window size
            train_size: float between 0 and 1
                The proportion of the dataset to include in the train set
            sum_: bool
                If true, the output will be the sum over the next 30 minutes   
        """
        
        if not hasattr(self, 'ts_dis'):
            print("Time series isn't discretized\n")
            self.discretize()
        
        n_train = int(train_size * len(self.ts_dis))

        if "CNN" in name:
            train = self.ts[:n_train]
            test = self.ts[n_train:]
        
        else:
            train = self.ts_dis[:n_train]
            test = self.ts_dis[n_train:]
            
        self.trainX, self.trainY = create_X_Y(train, f_size, offset, output_width, sum_)
        self.testX, self.testY = create_X_Y(test, f_size, offset, output_width, sum_)
        
        
    def get_models_size(self, models=["CNN_LSTM", "CNN", "LSTM", "MLP"], size=None):
        
        """
            Finds the optimal input window size for each model, stores the results in the conf dictionary
            
            @Params:
            models: array of string
                The names of the models used for the prediction, allowed names are 'CNN_LSTM', 'CNN', 'LSTM' and 'MLP'
            size: dict
                Optional dictionary containing the size for each model, keys correspond to the models names,
                if left None, the algorithm will find the optimal window size
        """
                
        self.conf["w_sizes"] = {}
        
        for model in models:
            
            print(f"Searching optimal window size for {model}:\n")
            
            if size:
                self.conf["w_sizes"][model] = size[model]
                
            else:
                self.conf["w_sizes"][model] = self.find_w_size(model, output_width=int(30/self.conf["time_step"]))
              
            
    def get_models(self, offset=0, sum_=False):
        
        """
            Creates and stores the models that will be used for predicting the time series
            according to their kind and their input window stored in the configuration dictionary
            Stores the models in a new dictionary
            For each model and for each predicted window, store the prediction in a new DataFrame named "predict"
            
            @Params:
            offset: int
                The offset between the last value of the input window and the first value of the output window
            sum_: bool
                If true, the output will be the sum over the next 30 minutes
        """
            
        self.models = {}
        self.predict = pd.DataFrame()
        min_value = min(self.conf["w_sizes"].values())
           
        output_width = int(30/self.conf["time_step"])
        
        
        for name in self.conf["w_sizes"].keys():
                
            size = self.conf["w_sizes"][name]
            self.create_train_test(name=name, f_size=size, offset=offset, output_width=output_width, sum_=sum_)
            model, loss, val_loss = get_model(name, self.trainX, self.trainY)
            
            pred = pd.DataFrame({name: model.predict(self.testX).tolist()},
                                index=range(size-min_value, len(self.testY)+(size-min_value)))
            
            pred[name] = pred[name].apply(lambda x: np.array(x))
            
            self.predict = pd.concat([self.predict, pred], axis=1)
                
            self.models[name] = model
            
            del model, pred
            
        self.create_train_test(name="CNN", f_size=min_value, offset=offset, output_width=output_width, sum_=sum_)
        self.predict["test"] = self.testY.tolist()
        self.create_train_test(name="MLP", f_size=min_value, offset=offset, output_width=output_width, sum_=sum_)
        self.predict["test_dis"] = self.testY.tolist()
        
        self.predict.dropna(inplace=True)
        
    
    def compute_error(self):
        
        """
            Computes the error between each model prediction for each window in the test set, stores the result
            in a new DataFrame. After this, creates a new column in the "predict" DataFrame with all errors in
            a numpy array
        """
        
        self.error = pd.DataFrame()
        
        for name in self.conf["w_sizes"].keys():
            
            self.error[f"mae {name}"] = self.predict[[name, "test"]].apply(lambda x: mae(x), axis=1)
            self.error[f"mape {name}"] = self.predict[[name, "test"]].apply(lambda x: MAPE(x[0], x[1]), axis=1)
            
        self.predict['error'] = self.error.filter(like='mae').apply(lambda r: tuple(r), axis=1).apply(np.array)
        
        
    def compute_weights_init(self, size_max=30, learning_rate=0.1):
        
        """
            Initializes the weights calculation and applies the function "compute_weights" 
            for each prediction in the "predict" DataFrame
            Stores the result in a new column in the "predict" DataFrame
            Computes the final time series with the model weights and their predictions
            Finally, computes the final error
            
            @Params:
            size_max: int
                The maximum size of the array containing the last errors to calculate the standard deviation
            learning_rate: float
                The value by which the new weights will be multiplied, it determines how
                newly acquired information will override old information, a value close to 1 
                will only consider new information while 0 will prevent it to learn
        
        """
        
        self.weights = [0] * len(self.models)
        self.last_values = pd.DataFrame()
        
        self.predict["weights"] = self.predict['error'].apply(lambda x: self.compute_weights(x))
        
        self.predict["final"] = self.predict[[*self.models.keys()]].apply(lambda x: self.compute_final_values(x), axis=1)
        
        self.error["mae final"] = self.predict[["final", "test"]].apply(lambda x: mae(x), axis=1)
        self.error["mape final"] = self.predict[["final", "test"]].apply(lambda x: MAPE(x[0], x[1]), axis=1)
        
    
    def compute_weights(self, error, size_max=30, learning_rate=0.1):
        
        """
            Computes the weight that each model will have for the prediction, a model which makes smaller errors and
            has a smaller standard deviation will have more weight for the prediction of the final values
            
            @Params:
            error: array of float
                The model errors used to update the weights
            size_max: int
                The maximum size of the array containing the last errors to calculate the standard deviation
            learning_rate: float
                The value by which the new weights will be multiplied, it determines how
                newly acquired information will override old information, a value close to 1 
                will only consider new information while 0 will prevent it to learn
                
            @Returns:
            weights: array of float
                The new updated weights
        """
        
        self.last_values = self.last_values.append([error], ignore_index=True)
        
        if len(self.last_values) > size_max:
            self.last_values = self.last_values.drop(self.last_values.index[0]).reset_index(drop=True)
    
        nw = error

        nw = [(max(nw)-elem+min(nw)) / sum(nw) for elem in nw]
        nw = [elem / sum(nw) for elem in nw]
    
        if np.array(self.weights).any():
            
            std = self.last_values.std().values
            std = [(max(std)-elem+min(std)) / sum(std) for elem in std]

            weights = np.add(self.weights, [learning_rate*a*b for a,b in zip(nw, std)])
            weights = [round((elem / sum(weights)), 2) for elem in weights]
            self.weights = weights
            
        else:
            self.weights = nw
            
        return np.array(self.weights)
    
    
    def compute_final_values(self, x):
        
        """
            Computes the final values with the model weights and their predictions
            
            @Params:
            x: Slice of DataFrame
                The DataFrame columns that contains the predictions of the models 
                
            @Returns:
            values: array of float
                The final window computed with the model weights and their predictions
        """
            
        values = np.zeros(len(x[0]))
    
        for i in range(len(x)):
            values = values + np.array(x.values[i] * self.weights[i])
        
        return values
    
    
    def plot_predictions(self, names=None, min_=1, max_=1000):
        
        """
            Plots the predicted windows for a given interval, some windows are skipped to avoid overlaps
            1 window displayed for 30/time_step predicted
            
            @Params:
            names: array of string
                If left None, will display every predictions + the original time series, "names" can be used
                to choose the predictions that must be plotted
            min_: int
                The lower bound index when plotting the time series
            max_: int
                The upper bound index when plotting the time series
        """
        
        if not names:
            names = [*self.models.keys()] + ["test", "final"]

        arr = range(min_, max_, int(30/self.conf["time_step"]))

        plt.figure(figsize=(16, 7), dpi=75)

        for name in names:
            plt.plot(np.concatenate(self.predict.iloc[arr][name].to_numpy()), label=name)

        plt.title("Predictions")
        plt.legend()
        plt.show()
    
    
    def plot_weights(self,):
        
        """
            Plots the weights evolution for each model used for the predictions
        """
        
        weights_evolution = pd.DataFrame(self.predict["weights"].values.tolist(), columns=[*self.models.keys()])

        plt.figure(figsize=(8, 5))

        for name in weights_evolution.columns:
            plt.plot(weights_evolution[name], label=name)

        plt.title("Weights evolution")
        plt.legend()
        plt.grid(axis="y", linestyle='--')
        plt.show()

        del weights_evolution
        

    def save_conf(self, name=None):
        
        """
            Saves a configuration as a json into a text file
            
            @Params:
            name: str
                The name that will be given to the file, if left None, the filename will be given according
                to the current date and the device code
        """
        
        if name:
            filename = name
            
        else:
            filename = "conf_" + str(self.conf["device"]) + "_" + datetime.today().strftime('%Y-%m-%d') + ".txt"
            
        with open(filename, "w") as file:
            json.dump(self.conf, file)
    
            
    def load_conf(self, filename):
        
        """
            Loads a configuration from a json stored in a text file
            
            @Params:
            filename: str
                The filename to load the configuration
        """
        
        with open(filename) as file:
            self.conf = json.loads(file.read())



# Data reading

Contains the main function to read files and load time series.

In [4]:

def create_dataframe(device_type=None, time=0, nbr_of_files=10000, show=True):
    
    """
        Reads the Excel files and create a pandas DataFrame containing 
        the time series information for a given device
        
        @Params:
        device_type: str
            String containing "0+int" between 0 and 8, which represent the code for the device type to be
            studied, if None, the loop will read every files for a complete dataset
        time: int 
            The timescale to use when reading the files, represents a time step in minutes
        nbr_of_files: int
            The number of files to be read
        show: bool
            If True, will display the informations while reading the files
            
        @Returns:
        df: DataFrame
            The DataFrame containing the informations from files for the given device
    """
    
    path = "../"

    df = pd.DataFrame()
    i = 1

    for root, directories, filenames in os.walk(path):
        for filename in filenames:
            if (filename.split(".")[-1] == "gzip") and (filename[0:2] == device_type or device_type == None):

                df2 = pd.read_parquet(os.path.join(root, filename)) 
                
                df2["date"] = df2["timestamp"].apply(lambda x: datetime.fromtimestamp(x/1000))
                
                if time != 0:   
                    df2 = df2.set_index('date').resample(f'{time}T').mean().reset_index()
                
                df2["filename"] = i
                df2["device"] = filename
                df = df.append(df2)
                if show: print("File {}, {} | {} rows".format(i, filename, len(df2)))
                i += 1
                
        if i > nbr_of_files:
            break
            
    df = df.drop_duplicates(subset=['date'])
    df = df.reset_index()

    return df



# Error helping functions

Functions used to compute or print the errors between time series / predicted windows.

In [5]:

def MAPE(y_true, y_pred):
    
    """
        Computes the MAPE between 2 time series
        
        @Params:
        y_true: array of float
            The original time series
        y_pred: array of float
            The predicted, interpolated or discretized time series
            
        @Returns:
        float
            The MAPE between the 2 time series
    """
    
    m = MeanAbsolutePercentageError()
    m.update_state(y_true, y_pred)
    
    return m.result().numpy()


def print_error(y_true, y_pred):
    
    """
        Prints the MSE, the MAE and the MAPE between 2 time series
        
        @Params:
        y_true: array of float
            The original time series
        y_pred: array of float
            The predicted, interpolated or discretized time series
    """
     
    print("\tMAPE: " + str(MAPE(y_true, y_pred)))
    print("\tMSE: " + str(mean_squared_error(y_true, y_pred)))
    print("\tMAE: " + str(mean_absolute_error(y_true, y_pred)))

    
def mae(x):
    
    """
        Computes and returns the MAE between a prediction window and the original window
            
        @Params:
        x: object
            Slice of a DataFrame with 2 columns, the one containing every predicted windows and the other with
            the original windows
            
        @Returns:
        float
            The MAE between the 2 value windows
    """

    return mean_absolute_error(x[0], x[1])



# Time step selection

Functions that are used to find the optimal time step for a given time series.

In [6]:

def check_timescale(df, kind="linear", graph=True, error=True):
    
    """
        Checks if the right timescale has been selected, computes the MAPE between the raw time series and
        a time series resampled at a given time step, interpolated to have the same length than the raw one
        
        @Params:
        df: DataFrame
            The DataFrame containing the time series information
        kind: str
            Kind of interpolation, default is linear but can be cubic..      
        graph: bool
            If True, plots the different graphs
        error: bool
            If True, prints the error
            
        @Returns
        float
            The MAPE between the raw time series and the interpolated one
    """
    
    raw = create_dataframe(device_type=df['device'].iloc[0][0:2], time=1, nbr_of_files=1, show=False)
    
    x = df[df["filename"].between(1, 4)].index
    y = df.iloc[x]["active_power"].to_numpy()
    
    step = len(raw) / len(x)
    x = (x*step).astype(int)
    
    f = interp1d(x, y, kind=kind)
    xnew = range(max(x))

    new = f(xnew)
    
    if graph:
        plt.scatter(x=range(0, 240, 2), y=y[:120], s=8, color="#4e3185")
        plt.title("Re-sampled time series")
        plt.show()

        plt.scatter(x=range(0, 240, 2), y=y[:120], s=8, color="#4e3185")
        plt.plot(new[:240], color="#5071ad", label="Interpolated")
        plt.legend()
        plt.title("Interpolated time series")
        plt.show()

        plt.plot(raw["active_power"][:240], color="#731135", label="Original")
        plt.scatter(x=range(0, 240, 2), y=y[:120], s=8, color="#4e3185")
        plt.plot(new[:240], color="#5071ad", label="Interpolated")
        plt.legend()
        plt.title("Re-sampled vs original")
        plt.show()

    if error: 
        print_error(raw["active_power"][0:max(x)], new)
    
    return MAPE(raw["active_power"][0:max(x)], new)
    del raw


def find_time_step(device_code, threshold=15):
    
    """
        Checks multiple time steps to find the optimal one, while the error (MAPE) is smaller than the threshold, 
        the time step to resample the time series increases. Time steps used are 1, 2, 5, 10, 15 minutes
        
        @Params:
        device_code: str
            String containing "0+int" between 0 and 8, which represents the code for the device type to be studied
        threshold: int
            The maximum value that the MAPE mustn't cross
            
        @Returns
        time_step: int
            The optimal computed time step
    """
    
    time_step = 1
    
    for i in [1, 2, 5, 10, 15]:
        
        print(f"Checking error with {i} min...")
        df = create_dataframe(device_code, i, 1, show=False)
        error = check_timescale(df, graph=False, error=False)
        print(f"\tMAPE : {error}\n")
        
        if error < threshold:
            time_step = i
        #else:
            #break
            
    print(f"Optimal time step is {time_step} min\n\n")  
    
    return time_step



# Clustering and discretization

The clustering and the discretization functions that will help to discretize the time series and plot the result.

In [7]:

def discretize_signal(labels, centroids):
    
    """
        Discretizes a time series using the labels computed with a clustering method and the centroids
        
        @Params:  
        labels: array of int
            The labels found with the clustering method
        centroids: array of float
            The centroid for each cluster
            
        @Returns:
        ts: array of float
            The discretized time series
    """
    
    function = lambda x: centroids[x]
    ts = np.array([function(xi) for xi in labels]).reshape(-1)
    
    return ts


def plot_discretized(raw, discretized, t1=0, t2=100, t1p=None, t2p=None):
    
    """
        Plots the discretized time series compared to the raw one
        
        @Params:
        raw: array of float
            The raw time series
        discretized: array of float 
            The discretized time series
        t1: int
            The lower bound index when plotting the time series
        t2: int
            The upper bound index when plotting the time series
        t1p: float between 0 and 1
            The lower bound index in percentage when plotting 
        t2p: float between 0 and 1
            The upper bound of time in percentage when plotting
    """
    
    if t1p and t2p:
        t1 = int(len(raw)*(t1p/100))
        t2 = int(len(raw)*(t2p/100))
    
    plt.plot(raw[t1:t2], color="#731135")
    plt.title("Original time series")
    plt.show()
    
    plt.plot(discretized[t1:t2], color="#5071ad")
    plt.title("Discretized time series")
    plt.show()
    
    plt.plot(raw[t1:t2], label="Original", color="#731135")
    plt.plot(discretized[t1:t2], label="Discretized", color="#5071ad")
    plt.title("Discretization")
    plt.legend()
    plt.show()

    
def clustering(x, n_clusters=3, error=True, plot=True):
    
    """
        Applies K-means clustering method to the time series according to the number of clusters given
        
        @Params:
        df: DataFrame
            The DataFrame containing the time series information
        n_clusters: int
            The number of clusters
        error: bool
            If True, prints the error
        plot: bool
            If True, plots the different graphs
            
        @Returns:
        ts: array of float
            The discretized time series
    """

    X = x.to_numpy().reshape(-1, 1)

    clustering = KMeans(n_clusters=n_clusters, random_state=0).fit(X)

    ts = discretize_signal(clustering.labels_, clustering.cluster_centers_)

    if plot: plot_discretized(X, ts, t1=0, t2=200)
    if error: print_error(X, ts)
    
    return ts



# Data windowing

Function that format and shape the time series into a dataset with input windows and output windows that can be used by Deep Learning models.

In [8]:

def create_X_Y(ts, f_size=10, offset=0, output_width=1, sum_=False):
    
    """
        From a given time series, creates an input array containing the input windows and a label array containing 
        the output windows for machine/deep learning tasks
        
        @Params:
        ts: array of float
            The time series
        f_size: int
            The input window size
        offset: int
            The offset between the last value of the input window and the first value of the output window
        output_width:
            The output window size
        sum_: bool
            If true, the label to predict will be the sum of the values contained in the output window
            
        @Returns:
        X: array of float
            The input array containing the input windows
        Y: array of float
            The label array containing the output windows
    """

    X, Y = [], []

    for i in range(len(ts) - f_size - offset - output_width):
        if (output_width == 1):
            Y.append(ts[i + f_size + offset])
        elif not sum_:
            Y.append(ts[i + f_size + offset:i + f_size + offset + output_width])
        else:
            Y.append(np.sum(ts[i + f_size + offset:i + f_size + offset + output_width]))
        X.append(ts[i:(i + f_size)])
    
    X, Y = np.array(X), np.array(Y)
    
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    if sum_: Y = np.reshape(Y, (Y.shape[0], 1))

    return X, Y



# Deep Learning models

Contains every Deep Learning models used for the hybrid prediction.

In [9]:

def get_model(name, trainX, trainY):
    
    if name == "CNN_LSTM":
        model, loss, val_loss = getLSTM_CNN(trainX, trainY)
        
    elif name == "CNN":
        model, loss, val_loss = getCNN(trainX, trainY)
        
    elif name == "LSTM":
        model, loss, val_loss = getLSTM(trainX, trainY)
        
    else:
        model, loss, val_loss = getMLP(trainX, trainY)
        
    return model, loss, val_loss
        

def getLSTM_CNN(trainX, trainY):
        
    model = Sequential([
        
        Conv1D(filters=60, kernel_size=5, strides=1, padding="causal", activation="relu"),
        
        LSTM(48, return_sequences=True),
        LSTM(48, return_sequences=False),
        
        Dense(32, activation='relu'),
        Dense(32, activation='relu'),
        
        Dense(trainY.shape[1], kernel_initializer=tf.initializers.zeros),
        Lambda(lambda x: x * 400)
        
    ])
    
    model.compile(loss="mae", optimizer="adam")
    
    history = model.fit(trainX, trainY, epochs=15, batch_size=32, validation_split=0.2)
    
    return model, np.array(history.history["loss"][-3:]).mean(), np.array(history.history["val_loss"][-3:]).mean()


def getCNN(trainX, trainY):
    
    model = Sequential([
        
        Conv1D(filters=64, kernel_size=3, strides=1, padding="causal", activation="relu"),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(32, activation='relu'),
        Dense(trainY.shape[1], kernel_initializer=tf.initializers.zeros),
        
    ])
    
    model.compile(optimizer='adam', loss='mae')

    history = model.fit(trainX, trainY, epochs=25, batch_size=32, validation_split=0.2)
    
    return model, np.array(history.history["loss"][-3:]).mean(), np.array(history.history["val_loss"][-3:]).mean()
    
    
def getMLP(trainX, trainY):
    
    model = Sequential([
        
        Flatten(),
        Dense(30, activation='relu'),
        Dense(30, activation='relu'),
        Dense(trainY.shape[1], kernel_initializer=tf.initializers.zeros),
        
    ])

    model.compile(optimizer='adam', loss='mae')

    history = model.fit(trainX, trainY, epochs=50, batch_size=32, validation_split=0.2)
    
    return model, np.array(history.history["loss"][-3:]).mean(), np.array(history.history["val_loss"][-3:]).mean()


def getLSTM(trainX, trainY):

    model = Sequential([
        
        LSTM(48, activation='relu', return_sequences=True),
        LSTM(48, activation='relu', return_sequences=False),
        Dense(32, activation='relu'),
        Dense(trainY.shape[1], kernel_initializer=tf.initializers.zeros),
    ])
    
    optimizer = Adam(learning_rate=0.003)
    model.compile(optimizer=optimizer, loss='mae')

    history = model.fit(trainX, trainY, epochs=30, batch_size=64, validation_split=0.2)
    
    return model, np.array(history.history["loss"][-3:]).mean(), np.array(history.history["val_loss"][-3:]).mean()



In [ ]:
ts = TimeSeries("05")

In [ ]:
ts.discretize()

In [ ]:
ts.get_models_size()

In [ ]:
ts.get_models()

In [ ]:
ts.compute_error()

In [ ]:
ts.compute_weights_init()

In [ ]:
ts.predict

In [ ]:
ts.plot_weights()

In [ ]:
ts.error

In [ ]:
ts.error.mean()

In [ ]:
ts.plot_predictions()

In [ ]:
ts.save_conf()